<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%94%D0%9E%D0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Динамические окрестностные модели (ДОМ)
####Вариант 5

In [23]:
import numpy as np
import pandas as pd
import random
import copy
import math
from prettytable import PrettyTable
import itertools as it

In [24]:
#A={a1, a2, a3}

#какие дуги входят в позицию:
#Ox[a1]={a1}
#Ox[a2]={}
#Ox[a3]={a1, a2}

#в какие позиции входят v
#Ov[a1]={a1}
#Ov[a2]={a2}
#Ov[a3]={a3}

#количества состояний/входов/выходов
elem = 100
count = [3, 3, 3]
Sx = np.array([[1, 0, 1], [0, 0, 1], [0, 0, 0]])
Sv = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

print("Матрица смежности:")
print("Sx = ")
print(Sx)
print("Sv = ")
print(Sv)

Матрица смежности:
Sx = 
[[1 0 1]
 [0 0 1]
 [0 0 0]]
Sv = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]


In [25]:
def x1(x, v):
  return math.sin(math.log(abs(v[0])) / x[0])

def x2(x, v):
  return math.cos(v[1]**3)

def x3(x, v):
  return math.sin(math.tan(x[0] / x[1]) - 3**(v[2]))

def v1(x, v):
  return (v[0] + math.tan(x[0])**2)

def v2(x, v):
  return (0.2 * v[1])

def v3(x, v):
  return (math.exp(x[0]) + .45 * math.log2(abs(x[1] * v[2])))

def y1(x, v):
  return (x[0] / v[0]) * math.sqrt(abs(x[0]))

def y2(x, v):
  return math.sqrt(abs(math.sin(v[1])))

def y3(x, v):
  return v[2] * x[0] / (x[1] + 0.1)

functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]

In [26]:
#Метод псевдообращения Бена-Израиля
def BenIsrael(A, eps = 0.000001): #А - np.array, eps - точность вычислительная
  I = np.diag(np.diag(np.ones([len(A),len(A)])))
  buf = 1
  i = 1

  if(np.linalg.norm(A, ord='fro') != 0): #если не нулевая норма
    gamma = 1.6 / (np.linalg.norm(A, ord='fro')**2)
    A1 = gamma * (A.T)
    i += 1
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
  else: #если нулевая норма
    A1 = np.zeros((len(A[0]), len(A)))
    buf = 0

  while(E >= eps and buf):
    A1 = A2.copy()
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
    i += 1

  return A1

#решение СЛАУ вида AX=B
def solvingSLAE(A, B):
  A = np.array(A.copy())

  #проверка на бесконечное количество решений
  A_psev = BenIsrael(A)

  X = A_psev.dot(B)
  return X

#=================================================================
#Генерация случайного вектора из 3-х элементов в диапазоне от -3 до 3
def generationRandomVector(elem = 3, start = -3, end = 3):
  vector = []
  for i in range(elem):
    vector.append(random.uniform(start, end))
  return vector

#=================================================================
#создание столбца таблицы
def createVariants(function, x, v, typeFunc = 1, elem = 100):
  vector = []

  if(typeFunc == 0):
    for i in range(elem):
      vector.append(function(generationRandomVector(), generationRandomVector()))
  else:
    for i in range(elem):
      vector.append(function(x[i], v[i]))

  return vector

#=================================================================
#Создание ДОМ в определённый момент времени
def createTableDNM(functions, count, step, oldTable = []): #count(x, v, y)
  table = []
  vector = []
  columns = createColumns(step, count)

  if(step == 0):
    for i in range(sum(count[0:2])):
      table.append(createVariants(functions[i], [], [], 0)) #xi, vi

  else:
    for i in range(count[0]): #xi
      table.append(createVariants(functions[i], (np.array(oldTable))[:, 0:count[0]], (np.array(oldTable))[:, count[0]:sum(count[0:2])]))
    for i in range(count[1]): #vi
      table.append(createVariants(functions[count[0] + i], [], [], 0))

  for i in range(count[2]): #yi
    table.append(createVariants(functions[sum(count[0:2]) + i], (np.array(table).T)[:, 0:count[0]], (np.array(table).T)[:, count[0]:sum(count[0:2])]))

  table = pd.DataFrame(np.array(table).T, columns = columns)
  return table

#заполнение
def createColumns(step, count):
  columns = []
  for i in range(count[0]):
    columns.append("x" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[1]):
    columns.append("v" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[2]):
    columns.append("y" + str(i + 1) + "(" + str(step) + ")")
  return columns

In [27]:
#functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]
model_0 = createTableDNM(functions, count, 0)
model_1 = createTableDNM(functions, count, 1, np.array(model_0))
model_2 = createTableDNM(functions, count, 2, np.array(model_1))
model_3 = createTableDNM(functions, count, 3, np.array(model_2))
model_4 = createTableDNM(functions, count, 4, np.array(model_3))
model_5 = createTableDNM(functions, count, 5, np.array(model_4))
model_6 = createTableDNM(functions, count, 6, np.array(model_5))
model_7 = createTableDNM(functions, count, 7, np.array(model_6))
model_8 = createTableDNM(functions, count, 8, np.array(model_7))
model_9 = createTableDNM(functions, count, 9, np.array(model_8))
model_10 = createTableDNM(functions, count, 10, np.array(model_9))

In [28]:
model_9

,x1(9),x2(9),x3(9),v1(9),v2(9),v3(9),y1(9),y2(9),y3(9)
0,0.984924,0.990444,-0.849010,-0.713531,0.066319,0.979271,-1.369908,0.257431,0.884509
1,0.016143,0.999999,0.318040,9.118164,-0.009871,0.397132,0.000225,0.099350,0.005828
2,-0.829393,1.000000,-0.551494,16.345650,-0.500792,-2.154296,-0.046210,0.692908,1.624325
3,0.066171,0.999928,0.274511,1.912145,0.398679,-0.073832,0.008902,0.623058,-0.004442
4,-0.949182,0.999999,0.817733,9.594270,0.509424,3.358455,-0.096386,0.698337,-2.897988
...,...,...,...,...,...,...,...,...,...
95,0.921926,0.999489,0.874032,-1.251963,0.045049,0.732305,-0.707055,0.212213,0.614041
96,-0.805786,0.995375,0.999705,2.315293,-0.554815,0.768095,-0.312409,0.725800,-0.565031
97,-0.448213,0.991621,0.272537,-1.240374,-0.369645,1.044552,0.241921,0.601069,-0.428887
98,-0.297903,0.998061,0.589788,0.444619,0.051640,-2.689068,-0.365700,0.227193,0.729543


In [29]:
model_10

,x1(10),x2(10),x3(10),v1(10),v2(10),v3(10),y1(10),y2(10),y3(10)
0,-0.336028,1.000000,-0.984407,178.528940,0.508609,-0.470837,-0.001091,0.697827,0.143831
1,-0.967662,1.000000,-0.999201,-1.710970,-0.503580,9.957519,0.556344,0.694668,-8.759556
2,0.225144,0.992123,-0.926833,5.552214,0.071533,2.400784,0.019241,0.267343,0.494927
3,-0.362911,0.997993,-0.755107,1.687337,-0.382800,1.138814,-0.129568,0.611162,-0.376403
4,-0.688462,0.991274,0.552778,-1.586316,0.139816,1.861836,0.360106,0.373310,-1.174594
...,...,...,...,...,...,...,...,...,...
95,0.241336,1.000000,-0.793053,-0.627847,0.128659,0.204109,-0.188834,0.358196,0.044781
96,-0.863357,0.985452,0.231044,32.141926,0.528370,2.754082,-0.024958,0.710018,-2.190567
97,-0.462314,0.998725,0.474523,-1.912657,0.340929,4.448409,0.164350,0.578241,-1.871773
98,0.408480,1.000000,-0.352080,6.167494,0.450875,0.098429,0.042330,0.660116,0.036551


##1. Линейная форма функции пересчёта

In [30]:
#Построение линейной регрессионной модели
def buildLinearRegretion(model1, model2, cols):
  vector_ones = [1] * elem
  estimate_linear = []
  variance_linear = []
  error_linear = []

  linearFunction = (model1.copy())[cols[0:(len(cols) - 1)]]
  linearFunction[cols[len(cols) - 1]] = (model2.copy())[cols[len(cols) - 1]]
  linearFunction.insert(0, "optional_col", vector_ones)

  A_linear = np.array(linearFunction)
  #A_psev = BenIsrael(A_linear[:, 0:(len(A_linear[0]) - 1)])
  X_linear = solvingSLAE(A_linear[:, 0:(len(A_linear[0]) - 1)], np.matrix(A_linear[:, len(A_linear[0]) - 1]).T)

  for i in range(len(linearFunction)):
    for j in range(0, len(X_linear)):
      if(j == 0):
        estimate_linear.append(X_linear[j,0])
      else:
        estimate_linear[i] += X_linear[j,0] * np.array(linearFunction)[i,j]

    variance_linear.append((1.0 / len(linearFunction)) * (np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])**2)
    #error_linear.append((1.0 / len(linearFunction)) * abs((np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])
                            #/ np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1]) * 100)
    error_linear.append((1.0 / len(linearFunction)) * abs((np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])
                            / max(np.array(linearFunction)[:, len(np.array(linearFunction)[0]) - 1])) * 100)

  linearFunction['regression'] = estimate_linear
  linearFunction['variance'] = variance_linear
  linearFunction['error, %'] = error_linear

  return X_linear, linearFunction

###Для функции $x_1[t+1]$

In [31]:
#x1, v1+
X1_linear, X1_linearTable = buildLinearRegretion(model_9, model_10, ['x1(9)', 'v1(9)', 'x1(10)'])
print(pd.DataFrame(X1_linear, columns=['X1'], index=['a1', 'c11', 'b11']))
X1_linearTable

           X1
a1  -0.060018
c11  0.348000
b11  0.000065


,optional_col,x1(9),v1(9),x1(10),regression,variance,"error, %"
0,1,0.984924,-0.713531,-0.336028,0.282690,0.003828,0.618758
1,1,0.016143,9.118164,-0.967662,-0.053810,0.008351,0.913911
2,1,-0.829393,16.345650,0.225144,-0.347589,0.003280,0.572769
3,1,0.066171,1.912145,-0.362911,-0.036867,0.001063,0.326065
4,1,-0.949182,9.594270,-0.688462,-0.389713,0.000893,0.298769
...,...,...,...,...,...,...,...
95,1,0.921926,-1.251963,0.241336,0.260732,0.000004,0.019397
96,1,-0.805786,2.315293,-0.863357,-0.340282,0.002736,0.523109
97,1,-0.448213,-1.240374,-0.462314,-0.216077,0.000606,0.246254
98,1,-0.297903,0.444619,0.408480,-0.163660,0.003273,0.572176


In [32]:
print("E = " + str(sum(X1_linearTable['variance'])))
print("A = " + str(sum(X1_linearTable['error, %'])))

E = 0.4576242556722923
A = 59.14357203001952


###Для функции $x_2[t+1]$

In [33]:
#v2+
X2_linear, X2_linearTable = buildLinearRegretion(model_9, model_10, ['v2(9)', 'x2(10)'])
print(pd.DataFrame(X2_linear, columns=['X2'], index=['a2', 'b22']))
X2_linearTable

           X2
a2   0.996111
b22  0.003026


,optional_col,v2(9),x2(10),regression,variance,"error, %"
0,1,0.066319,1.000000,0.996312,1.359980e-07,0.003688
1,1,-0.009871,1.000000,0.996082,1.535386e-07,0.003918
2,1,-0.500792,0.992123,0.994596,6.114160e-08,0.002473
3,1,0.398679,0.997993,0.997318,4.555586e-09,0.000675
4,1,0.509424,0.991274,0.997653,4.069212e-07,0.006379
...,...,...,...,...,...,...
95,1,0.045049,1.000000,0.996248,1.407897e-07,0.003752
96,1,-0.554815,0.985452,0.994432,8.064863e-07,0.008980
97,1,-0.369645,0.998725,0.994993,1.392732e-07,0.003732
98,1,0.051640,1.000000,0.996268,1.392967e-07,0.003732


In [34]:
print("E = " + str(sum(X2_linearTable['variance'])))
print("A = " + str(sum(X2_linearTable['error, %'])))

E = 3.554859590272665e-05
A = 0.4659644804885926


###Для функции $x_3[t+1]$

In [35]:
#x1, x2, v3+
X3_linear, X3_linearTable = buildLinearRegretion(model_9, model_10, ['x1(9)', 'x2(9)', 'v3(9)', 'x3(10)'])
print(pd.DataFrame(X3_linear, columns=['X3'], index=['a3', 'c31', 'c32', 'b33']))
X3_linearTable

           X3
a3   8.697982
c31  0.022757
c32 -8.818619
b33  0.019702


,optional_col,x1(9),x2(9),v3(9),x3(10),regression,variance,"error, %"
0,1,0.984924,0.990444,0.979271,-0.984407,0.005339,0.009796,0.990630
1,1,0.016143,0.999999,0.397132,-0.999201,-0.112439,0.007863,0.887554
2,1,-0.829393,1.000000,-2.154296,-0.926833,-0.181954,0.005548,0.745544
3,1,0.066171,0.999928,-0.073832,-0.755107,-0.119947,0.004034,0.635726
4,1,-0.949182,0.999999,3.358455,0.552778,-0.076064,0.003954,0.629403
...,...,...,...,...,...,...,...,...
95,1,0.921926,0.999489,0.732305,-0.793053,-0.080724,0.005074,0.712964
96,1,-0.805786,0.995375,0.768095,0.231044,-0.083055,0.000987,0.314380
97,1,-0.448213,0.991621,1.044552,0.474523,-0.036362,0.002610,0.511341
98,1,-0.297903,0.998061,-2.689068,-0.352080,-0.163292,0.000356,0.188956


In [36]:
print("E = " + str(sum(X3_linearTable['variance'])))
print("A = " + str(sum(X3_linearTable['error, %'])))

E = 0.4696865200540745
A = 62.05551547767458


###Для функции $y_1[t]$

In [37]:
#x1, v1+
Y1_linear, Y1_linearTable = buildLinearRegretion(model_9, model_9, ['x1(9)', 'v1(9)', 'y1(9)'])
print(pd.DataFrame(Y1_linear, columns=['Y1'], index=['a1', 'c11', 'b11']))
Y1_linearTable

           Y1
a1  -0.198663
c11  0.183577
b11  0.000063


,optional_col,x1(9),v1(9),y1(9),regression,variance,"error, %"
0,1,0.984924,-0.713531,-1.369908,-0.017898,0.018279,0.168503
1,1,0.016143,9.118164,0.000225,-0.195129,0.000382,0.024347
2,1,-0.829393,16.345650,-0.046210,-0.349898,0.000922,0.037849
3,1,0.066171,1.912145,0.008902,-0.186396,0.000381,0.024340
4,1,-0.949182,9.594270,-0.096386,-0.372311,0.000761,0.034389
...,...,...,...,...,...,...,...
95,1,0.921926,-1.251963,-0.707055,-0.029497,0.004591,0.084445
96,1,-0.805786,2.315293,-0.312409,-0.346442,0.000012,0.004242
97,1,-0.448213,-1.240374,0.241921,-0.281022,0.002735,0.065175
98,1,-0.297903,0.444619,-0.365700,-0.253324,0.000126,0.014006


In [38]:
print("E = " + str(sum(Y1_linearTable['variance'])))
print("A = " + str(sum(Y1_linearTable['error, %'])))

E = 2.090337669317277
A = 8.519203072057843


###Для функции $y_2[t]$

In [39]:
#v2+
Y2_linear, Y2_linearTable = buildLinearRegretion(model_9, model_9, ['v2(9)', 'y2(9)'])
print(pd.DataFrame(Y2_linear, columns=['Y2'], index=['a2', 'b22']))
Y2_linearTable

           Y2
a2   0.519913
b22 -0.054754


,optional_col,v2(9),y2(9),regression,variance,"error, %"
0,1,0.066319,0.257431,0.516282,0.000670,0.344552
1,1,-0.009871,0.099350,0.520453,0.001773,0.560524
2,1,-0.500792,0.692908,0.547333,0.000212,0.193772
3,1,0.398679,0.623058,0.498084,0.000156,0.166351
4,1,0.509424,0.698337,0.492020,0.000426,0.274624
...,...,...,...,...,...,...
95,1,0.045049,0.212213,0.517446,0.000932,0.406291
96,1,-0.554815,0.725800,0.550291,0.000308,0.233617
97,1,-0.369645,0.601069,0.540152,0.000037,0.081086
98,1,0.051640,0.227193,0.517085,0.000840,0.385871


In [40]:
print("E = " + str(sum(Y2_linearTable['variance'])))
print("A = " + str(sum(Y2_linearTable['error, %'])))

E = 0.031109283631558175
A = 19.791071293506896


###Для функции $y_3[t]$

In [41]:
#x1, x2, v3+
Y3_linear, Y3_linearTable = buildLinearRegretion(model_9, model_9, ['x1(9)', 'x2(9)', 'v3(9)', 'y3(9)'])
print(pd.DataFrame(Y3_linear, columns=['Y3'], index=['a3', 'c31', 'c32', 'b33']))
Y3_linearTable

            Y3
a3  -58.401012
c31   2.101467
c32  58.707044
b33  -0.109890


,optional_col,x1(9),x2(9),v3(9),y3(9),regression,variance,"error, %"
0,1,0.984924,0.990444,0.979271,0.884509,1.707212,0.006768,0.080787
1,1,0.016143,0.999999,0.397132,0.005828,0.296271,0.000844,0.028521
2,1,-0.829393,1.000000,-2.154296,1.624325,-1.200174,0.079778,0.277358
3,1,0.066171,0.999928,-0.073832,-0.004442,0.448952,0.002056,0.044522
4,1,-0.949182,0.999999,3.358455,-2.897988,-2.057746,0.007060,0.082510
...,...,...,...,...,...,...,...,...
95,1,0.921926,0.999489,0.732305,0.614041,2.132964,0.023071,0.149154
96,1,-0.805786,0.995375,0.768095,-0.565031,-1.743225,0.013881,0.115696
97,1,-0.448213,0.991621,1.044552,-0.428887,-1.242595,0.006621,0.079904
98,1,-0.297903,0.998061,-2.689068,0.729543,-0.138357,0.007533,0.085225


In [42]:
print("E = " + str(sum(Y3_linearTable['variance'])))
print("A = " + str(sum(Y3_linearTable['error, %'])))

E = 6.360786295915852
A = 15.904161457802655


##2. Квадратичная форма функции пересчёта

In [42]:
#Построение квадратичной регрессионной модели
def buildQuadraticRegretion(model1, model2, cols):
  vector_ones = [1] * elem
  estimate = []
  variance = []
  error = []

  #сформировать столбцы из квадратов и 2х произведений переменных
  #...

  #переменные
  Function = (model1.copy())[cols[0:(len(cols) - 1)]]


  #значение функции
  Function[cols[len(cols) - 1]] = (model2.copy())[cols[len(cols) - 1]]

  #доп. столбец
  Function.insert(0, "optional_col", vector_ones)

  A = np.array(Function)
  #A_psev = BenIsrael(A[:, 0:(len(A[0]) - 1)])
  X = solvingSLAE(A[:, 0:(len(A[0]) - 1)], np.matrix(A[:, len(A[0]) - 1]).T)

  for i in range(len(Function)):
    for j in range(0, len(X)):
      if(j == 0):
        estimate.append(X[j,0])
      else:
        estimate[i] += X[j,0] * np.array(Function)[i,j]

    variance.append((1.0 / len(Function)) * (np.array(Function)[i, len(np.array(Function)[0]) - 1] - estimate[i])**2)
    #error_linear.append((1.0 / len(Function)) * abs((np.array(Function)[i, len(np.array(Function)[0]) - 1] - estimate[i])
                            #/ np.array(Function)[i, len(np.array(Function)[0]) - 1]) * 100)
    error.append((1.0 / len(Function)) * abs((np.array(Function)[i, len(np.array(Function)[0]) - 1] - estimate[i])
                            / max(np.array(Function)[:, len(np.array(Function)[0]) - 1])) * 100)

  Function['regression'] = estimate
  Function['variance'] = variance
  Function['error, %'] = error

  return X, Function

###Для функции $x_1[t + 1]$